In [28]:
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
import datetime
import warnings
warnings.filterwarnings("ignore")

# Usage Example

1.User read data for training:

`raw_data = pd.read_csv("user_file_path",index_col=[0])`

2.User read data for prediction:

`expetion_data = pd.read_csv("user_file_path",index_col=[0])`

3.User do training data wrangling and prediction data wrangling:

`train = training_wrangling(raw_data)`

`test = prediction_wrangling(expetion_data)`

4.User do predict exception using `train` and `test`, then can see the result dataframe:

`exception_prediction(train,test)`

In [29]:
# read dataframe
raw_data = pd.read_csv("exception_hours.csv",index_col=[0])
training = raw_data[(raw_data["SHIFT_DATE"]>"2012-12-31") & (raw_data["SHIFT_DATE"]<"2018-01-01")]
test = raw_data[(raw_data["SHIFT_DATE"]>"2017-12-31") & (raw_data["SHIFT_DATE"]<"2019-01-01")]

# Data Wrangling for Training Set 

In [33]:
def training_wrangling(dataframe):
    '''
    Given a dataframe, wrangling to format using for training the model.
    
    Paramenters
    -----------
    dataframe: dataframe
        Raw data 
        
    Returns
    -------
    dataframe
        One for natural prediction, one for conservative prediction
    '''
    df = dataframe.copy()
    
    # filter target group: Nurse
    df = df[df["LABOR_AGREEMENT"]=="NURS"]

    # filter the 6 site the client suggested
    df = df[(df["SITE"]=="St Paul's Hospital") |
            (df["SITE"]=="Mt St Joseph") |
            (df["SITE"]=="Holy Family") |
            (df["SITE"]=="SVH Langara") |
            (df["SITE"]=="Brock Fahrni") |
            (df["SITE"]=="Youville Residence")]
    
    # filter NaN in MIN_CALL_TO_MAX_CALL_MINUTES and EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES
    df = df[pd.notnull(df["MIN_CALL_TO_MAX_CALL_MINUTES"])]
    df = df[pd.notnull(df["EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES"])]

    # filter MIN_CALL_TO_MAX_CALL_MINUTES > 0 which means the call time is after exception creation time
    df = df[(df["MIN_CALL_TO_MAX_CALL_MINUTES"] >= 0)]

    # filter EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES < 0 which means the creation time is ahead of shift start
    df = df[(df["EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES"] <= 0)]

    # create NOTICE = EXCEPTION_CREATION_TO_MAXCALL_MINUTES - MIN_CALL_TO_MAX_CALL_MINUTES 
    #               =EXCEPTION_CREATION_TO_MINCALL_MINUTES
    df["NOTICE_"] = df["EXCEPTION_CREATION_TO_MAXCALL_MINUTES"] + df["MIN_CALL_TO_MAX_CALL_MINUTES"]

    # create a column to indicate the month of the SHIFT_DATE
    df["MONTH_"] = pd.to_datetime(df['SHIFT_DATE']).dt.month

    # create a column to indicate the SHIFT of the exception
    df["START_TIME"] = pd.to_datetime(df["START_TIME"])
    df["SHIFT_"] = 3
    df["SHIFT_"][(df["START_TIME"] >= "06:00:00") & (df["START_TIME"] < "13:00:00")] = 1
    df["SHIFT_"][(df["START_TIME"] >= "13:00:00") & (df["START_TIME"] < "19:00:00")] = 2

    # convert "EXCEPTION_GROUP","PROGRAM","SITE","DEPARTMENT" value from str to numeric for randomforest model
    le = preprocessing.LabelEncoder()
    df["EXCEPTION_GROUP_"] = le.fit_transform(df["EXCEPTION_GROUP"])                     
    df["PROGRAM_"] = le.fit_transform(df["PROGRAM"])
    df["SITE_"] = le.fit_transform(df["SITE"])
    df["JOB_FAMILY_"] = le.fit_transform(df["JOB_FAMILY"])
    df["SUB_PROGRAM_"] = le.fit_transform(df["SUB_PROGRAM"])
    df["DEPARTMENT_"] = le.fit_transform(df["DEPARTMENT"])
    return df

In [34]:
df2 = training_wrangling(training)
df2.head(10)

,PCN,EXCEPTION_REASON,EXCEPTION_GROUP,EXCEPTION_HOURS,EXCEPTION_CREATION_DATE,MASKED_REPLACED_EMPLID,LABOR_AGREEMENT,UNION_CD,JOB_FAMILY,JOB_FAMILY_DESCRIPTION,...,SCHEDULER_JOBCODE,NOTICE_,MONTH_,SHIFT_,EXCEPTION_GROUP_,PROGRAM_,SITE_,JOB_FAMILY_,SUB_PROGRAM_,DEPARTMENT_
EXCEPTIONID,,,,,,,,,,,,,,,,,,,,,
10440668,2764,PVC- Vacation Regular - MV- Move,Other,11.25,2016-12-21 09:34:15.627,8715983,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900440.0,-70889.0,4,1,5,1,4,5,14,33
9907772,6368,MV- Move - MV- Move,Move,11.25,2016-04-22 09:46:19.637,8710144,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,0.0,4,3,3,1,4,5,20,50
10820790,4907,USK- Unpd Sick Lv,Unpaid Sick,1.50,2017-06-14 12:16:46.433,8716743,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900964.0,-77.0,6,2,10,17,2,5,36,37
10370965,6464,UGL- Unpd LOA,Leave of Absence,1.50,2016-11-28 08:03:16.627,8710011,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,0.0,12,2,2,2,4,5,24,6
8097952,7434,Vacant Shift,Vacancy,7.50,2013-11-22 10:59:25.340,80,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,NaN,0.0,12,1,11,16,4,5,33,81
8738807,2923,PVC- Vacation Regular - MV- Move,Other,11.00,2014-10-27 14:55:45.493,8700997,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-66111.0,8,3,5,13,4,5,42,69
10746330,NaN,REG- Regular Hrs - MV- Move,Other,9.00,2017-05-08 10:16:22.360,BOOK-ON,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,0.0,6,1,5,2,4,5,24,6
9497491,2352,PVC- Vacation Regular - MV- Move,Other,7.50,2015-10-28 09:48:38.240,8705510,NURS,BCNU-Nur,DC2A00,Registered Nurse-DC2A Sup,...,900963.0,-130894.0,2,1,5,15,0,6,45,108
10909941,3583,CS- Casual Sick,Casual Sick or Cancelled,11.25,2017-07-28 20:06:15.423,8704152,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900964.0,-29.0,7,1,0,1,4,5,14,33


# Data Wrangling for Prediction Set

In [35]:
def prediction_wrangling(dataframe):
    '''
    Given a dataframe using for prediction, wrangling to format using for training the model.
    
    Paramenters
    -----------
    dataframe: dataframe
        Raw data of prediction, which should not contains "EARNING_CATEGORY"
        
    Returns
    -------
    dataframe
        Dataframe that contains converted value
        
    '''
    df = dataframe.copy()
    
    # if the dataframe contains EARNING_CATEGORY, drop the column
    if 'EARNING_CATEGORY' in df.columns:
        df = df.drop('EARNING_CATEGORY', 1)
    
    # filter target group: Nurse
    df = df[df["LABOR_AGREEMENT"]=="NURS"]

    # filter the 6 site the client suggested
    df = df[(df["SITE"]=="St Paul's Hospital") |
            (df["SITE"]=="Mt St Joseph") |
            (df["SITE"]=="Holy Family") |
            (df["SITE"]=="SVH Langara") |
            (df["SITE"]=="Brock Fahrni") |
            (df["SITE"]=="Youville Residence")]
    
    # filter NaN in MIN_CALL_TO_MAX_CALL_MINUTES and EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES
    # df["NOTICE_"] = pd.to_datetime(df["EXCEPTION_CREATION_DATE"]).map(lambda x:int((x - datetime.datetime.now()).total_seconds()/60))
    
    # filter NaN in MIN_CALL_TO_MAX_CALL_MINUTES and EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES
    df = df[pd.notnull(df["MIN_CALL_TO_MAX_CALL_MINUTES"])]
    df = df[pd.notnull(df["EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES"])]
    # filter MIN_CALL_TO_MAX_CALL_MINUTES > 0 which means the call time is after exception creation time
    df = df[(df["MIN_CALL_TO_MAX_CALL_MINUTES"] >= 0)]
    # filter EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES < 0 which means the creation time is ahead of shift start
    df = df[(df["EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES"] <= 0)]
    
    df["NOTICE_"] = df["EXCEPTION_CREATION_TO_MAXCALL_MINUTES"] + df["MIN_CALL_TO_MAX_CALL_MINUTES"]
    
    # create a column to indicate the month of the SHIFT_DATE
    df["MONTH_"] = pd.to_datetime(df['SHIFT_DATE']).dt.month

    # create a column to indicate the SHIFT of the exception
    df["START_TIME"] = pd.to_datetime(df["START_TIME"])
    df["SHIFT_"] = 3
    df["SHIFT_"][(df["START_TIME"] >= "06:00:00") & (df["START_TIME"] < "13:00:00")] = 1
    df["SHIFT_"][(df["START_TIME"] >= "13:00:00") & (df["START_TIME"] < "19:00:00")] = 2

    # convert "EXCEPTION_GROUP","PROGRAM","SITE","DEPARTMENT" value from str to numeric for randomforest model
    le = preprocessing.LabelEncoder()
    df["EXCEPTION_GROUP_"] = le.fit_transform(df["EXCEPTION_GROUP"])                     
    df["PROGRAM_"] = le.fit_transform(df["PROGRAM"])
    df["SITE_"] = le.fit_transform(df["SITE"])
    df["JOB_FAMILY_"] = le.fit_transform(df["JOB_FAMILY"])
    df["SUB_PROGRAM_"] = le.fit_transform(df["SUB_PROGRAM"])
    df["DEPARTMENT_"] = le.fit_transform(df["DEPARTMENT"])
    return df

In [36]:
df3 = prediction_wrangling(test)
df3.head(20)

,PCN,EXCEPTION_REASON,EXCEPTION_GROUP,EXCEPTION_HOURS,EXCEPTION_CREATION_DATE,MASKED_REPLACED_EMPLID,LABOR_AGREEMENT,UNION_CD,JOB_FAMILY,JOB_FAMILY_DESCRIPTION,...,SCHEDULER_JOBCODE,NOTICE_,MONTH_,SHIFT_,EXCEPTION_GROUP_,PROGRAM_,SITE_,JOB_FAMILY_,SUB_PROGRAM_,DEPARTMENT_
EXCEPTIONID,,,,,,,,,,,,,,,,,,,,,
11848782,11806,PSK- Sick Lv,Paid Sick,11.00,2018-09-13 10:01:37.393,8703044,NURS,BCNU-LPNFc,LPN001,Licensed Practical Nurse,...,900963.0,-5.0,9,1,6,10,4,9,38,64
11598682,7443,PSK- Sick Lv - CU- Cancelled by Unit,Paid Sick,10.00,2018-05-12 08:05:10.077,8716544,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900964.0,-5.0,5,1,6,1,2,3,12,31
11748260,3362,MV- Move,Move,7.50,2018-07-23 11:18:56.627,8715337,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-3.0,8,2,3,0,1,3,37,32
12085671,6529,CC- Casual Cancelled,Casual Sick or Cancelled,11.00,2018-12-06 11:04:44.240,8719321,NURS,HSA-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-19.0,12,3,0,13,4,3,35,86
11751544,2831,USK- Unpd Sick Lv,Unpaid Sick,10.00,2018-07-24 11:32:21.047,8700774,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-1276.0,9,1,10,6,4,3,4,51
11792180,8213,Vacant Shift,Vacancy,11.25,2018-08-14 10:22:18.613,80,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-60535.0,11,1,11,1,4,3,13,31
11378799,4383,UGL- Unpd LOA - CU- Cancelled by Unit,Leave of Absence,1.50,2018-02-02 13:47:27.440,8708236,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-1.0,2,1,2,10,4,3,39,15
11457156,NaN,AC- Adjusted On Call,Schedule Adjustment,14.00,2018-03-11 10:37:42.783,8705289,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900683.0,0.0,3,2,8,12,2,3,25,72
11442326,NaN,Vacant Shift,Vacancy,9.50,2018-03-05 09:24:11.883,80,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,900963.0,-11.0,3,1,11,2,4,3,23,7


# Exception Prediction

In [37]:
CAT_1 = ["Regular Relief Utilized",
         "Casual at Straight-Time",
         "PT Over FTE",
         "Miscellaneous Straight-Time",
         "PT Employee Moved - Straight-Time",
         "FT Employee Moved - Straight-Time"]
CAT_2 = ["Overtime",
         "Relief Not Found",
         "Agency",
         "Insufficient Notice",
         "On-Call"]

# create function for grouping label
def replace_str(string):
    if string in CAT_1:
        return string.replace(string, "Straight Time")
    elif string in CAT_2:
        return string.replace(string, "Overtime and Beyond")
    else:
        return string

def exception_prediction(dataframe1,dataframe2):
    '''
    Given training dataframe and prediction dataframe doing prediction.
    
    Paramenters
    -----------
    dataframe1: 
        dataframe wrangled by training_wrangling
    
    dataframe2: 
        dataframe wrangled by prediction_wrangling
        
    Returns
    -------
    dataframe
        Contains prediction result and Suggestion
    '''
    # create dataframe for Natural Prediction and Conservative Prediction
    df = dataframe1.copy()
    df["EARNING_CATEGORY"] = df["EARNING_CATEGORY"].apply(replace_str)

    # Prepare data for model fitting
    feature_cols = ["EXCEPTION_HOURS",
                    "EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES",
                    "SITE_",
                    "EXCEPTION_GROUP_",
                    "PROGRAM_",
                    "MONTH_",
                    "SUB_PROGRAM_",
                    "DEPARTMENT_",
                    "NOTICE_",
                    "SHIFT_"]
    X = df.loc[:, feature_cols]
    y = df.EARNING_CATEGORY

    # build random forest model and test
    RF = RandomForestClassifier(n_estimators=25, 
                                max_depth=15,
                                min_samples_split=6,
                                min_samples_leaf = 7,
                                class_weight='balanced')
    RF.fit(X,y)
    print("Model Training Score:", round(RF.score(X,y),3))

    # create result dataframe
    pred_dict = dataframe2.copy()
    pred_dict['PREDICTION'] = RF.predict(dataframe2.loc[:,feature_cols])
    result = pred_dict.drop(["SITE_", "EXCEPTION_GROUP_", "PROGRAM_", "MONTH_", "SUB_PROGRAM_", "DEPARTMENT_", "NOTICE_","JOB_FAMILY_"], axis=1)
    return result

In [38]:
df4 = exception_prediction(df2,df3)
df4.head()

Model Training Score: 0.85


,PCN,EXCEPTION_REASON,EXCEPTION_GROUP,EXCEPTION_HOURS,EXCEPTION_CREATION_DATE,MASKED_REPLACED_EMPLID,LABOR_AGREEMENT,UNION_CD,JOB_FAMILY,JOB_FAMILY_DESCRIPTION,...,EXCEPTION_CREATION_TO_FILL_MINUTES,EXCEPTION_CREATION_TO_MAXCALL_MINUTES,EXCEPTION_CREATION_TO_SHIFTSTART_MINUTES,EARL_NOTIFICATION_TO_FILL_MINUTES,EARL_NOTIFICATION_TO_SHIFTSTART_MINUTES,FILL_TO_SHIFTSTART_MINUTES,MASKED_SCHEDULER_EMPLID,SCHEDULER_JOBCODE,SHIFT_,PREDICTION
EXCEPTIONID,,,,,,,,,,,,,,,,,,,,,
11848782,11806,PSK- Sick Lv,Paid Sick,11.0,2018-09-13 10:01:37.393,8703044,NURS,BCNU-LPNFc,LPN001,Licensed Practical Nurse,...,0,-5733,-11339,0,-11349,0,828348.0,900963.0,1,Straight Time
11598682,7443,PSK- Sick Lv - CU- Cancelled by Unit,Paid Sick,10.0,2018-05-12 08:05:10.077,8716544,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,-86,-62,-175,-157,-246,89,842333.0,900964.0,1,Overtime and Beyond
11748260,3362,MV- Move,Move,7.5,2018-07-23 11:18:56.627,8715337,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,-1341,-1213,-13182,0,0,11841,825556.0,900963.0,2,Straight Time
12085671,6529,CC- Casual Cancelled,Casual Sick or Cancelled,11.0,2018-12-06 11:04:44.240,8719321,NURS,HSA-Nur,DC1000,Registered Nurse-DC1,...,-155,-19,-3386,0,0,3231,824333.0,900963.0,3,Straight Time
11751544,2831,USK- Unpd Sick Lv,Unpaid Sick,10.0,2018-07-24 11:32:21.047,8700774,NURS,BCNU-Nur,DC1000,Registered Nurse-DC1,...,-1347,-1347,-76138,0,0,74791,842235.0,900963.0,1,Straight Time


In [39]:
df4.to_csv ('classification_result.csv', index = None, header=True)
#Don't forget to add '.csv' at the end of the path